In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
import pdb

torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True) 
from torch.utils.tensorboard import SummaryWriter


In [2]:
import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)
import tensorboard
print(tensorboard.__version__)


1.1.0
0.3.0
1.15.0a20190806


In [3]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

In [4]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()


In [5]:
class Network(nn.Module):
    def __init__(self):
        super(Network,self).__init__()
        self.conv1 = nn.Conv2d(in_channels =1,out_channels =6,kernel_size =5)
        self.conv2 = nn.Conv2d(in_channels =6,out_channels =12,kernel_size =5)
        
        self.fc1 = nn.Linear(in_features = 12*4*4, out_features = 120)
        self.fc2 = nn.Linear(in_features = 120, out_features = 60)
        self.out = nn.Linear(in_features= 60, out_features=10)
        
    def forward(self, t):
    # (1) input layer
        t = t

    # (2) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

    # (3) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

    # (4) hidden linear layer
        t = t.reshape(-1, 12 * 4 * 4)
        t = self.fc1(t)
        t = F.relu(t)

    # (5) hidden linear layer
        t = self.fc2(t)
        t = F.relu(t)

    # (6) output layer
        t = self.out(t)
    #t = F.softmax(t, dim=1)

        return t

In [6]:
from itertools import product

In [7]:
parameters = dict(lr = [.01, .001, .0001, .00001],batch_size = [10,100, 1000],shuffle = [True, False])


In [8]:
param_values = [v for v in parameters.values()]
param_values

[[0.01, 0.001, 0.0001, 1e-05], [10, 100, 1000], [True, False]]

In [9]:
for lr, batch_size, shuffle in product(*param_values): 
    print (lr, batch_size, shuffle)
    network = Network()

    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
    optimizer = optim.Adam(network.parameters(), lr=lr)

    images, labels = next(iter(train_loader))
    grid = torchvision.utils.make_grid(images)

    comment=f' batch_size={batch_size} lr={lr}'
    tb = SummaryWriter(comment=comment)
    tb.add_image('images', grid)
    tb.add_graph(network, images)

    for epoch in range(5):
        total_loss = 0
        total_correct = 0
        for batch in train_loader:
            images, labels = batch # Get Batch
            preds = network(images) # Pass Batch
            loss = F.cross_entropy(preds, labels) # Calculate Loss
            optimizer.zero_grad() # Zero Gradients
            loss.backward() # Calculate Gradients
            optimizer.step() # Update Weights

            total_loss += loss.item() * batch_size
            total_correct += get_num_correct(preds, labels)

        tb.add_scalar('Loss', total_loss, epoch)
        tb.add_scalar('Number Correct', total_correct, epoch)
        tb.add_scalar('Accuracy', total_correct / len(train_set), epoch)

        for name, param in network.named_parameters():
            tb.add_histogram(name, param, epoch)
            tb.add_histogram(f'{name}.grad', param.grad, epoch)

            print(
                "epoch", epoch
                ,"total_correct:", total_correct
                ,"loss:", total_loss
            )  
tb.close()
        
        

0.01 10 True
epoch 0 total_correct: 44757 loss: 40776.29427675158
epoch 0 total_correct: 44757 loss: 40776.29427675158
epoch 0 total_correct: 44757 loss: 40776.29427675158
epoch 0 total_correct: 44757 loss: 40776.29427675158
epoch 0 total_correct: 44757 loss: 40776.29427675158
epoch 0 total_correct: 44757 loss: 40776.29427675158
epoch 0 total_correct: 44757 loss: 40776.29427675158
epoch 0 total_correct: 44757 loss: 40776.29427675158
epoch 0 total_correct: 44757 loss: 40776.29427675158
epoch 0 total_correct: 44757 loss: 40776.29427675158
epoch 1 total_correct: 47519 loss: 34585.23965639062
epoch 1 total_correct: 47519 loss: 34585.23965639062
epoch 1 total_correct: 47519 loss: 34585.23965639062
epoch 1 total_correct: 47519 loss: 34585.23965639062
epoch 1 total_correct: 47519 loss: 34585.23965639062
epoch 1 total_correct: 47519 loss: 34585.23965639062
epoch 1 total_correct: 47519 loss: 34585.23965639062
epoch 1 total_correct: 47519 loss: 34585.23965639062
epoch 1 total_correct: 47519 loss

epoch 1 total_correct: 50998 loss: 24236.041921377182
epoch 1 total_correct: 50998 loss: 24236.041921377182
epoch 1 total_correct: 50998 loss: 24236.041921377182
epoch 1 total_correct: 50998 loss: 24236.041921377182
epoch 1 total_correct: 50998 loss: 24236.041921377182
epoch 1 total_correct: 50998 loss: 24236.041921377182
epoch 1 total_correct: 50998 loss: 24236.041921377182
epoch 1 total_correct: 50998 loss: 24236.041921377182
epoch 1 total_correct: 50998 loss: 24236.041921377182
epoch 1 total_correct: 50998 loss: 24236.041921377182
epoch 2 total_correct: 51937 loss: 21929.36307489872
epoch 2 total_correct: 51937 loss: 21929.36307489872
epoch 2 total_correct: 51937 loss: 21929.36307489872
epoch 2 total_correct: 51937 loss: 21929.36307489872
epoch 2 total_correct: 51937 loss: 21929.36307489872
epoch 2 total_correct: 51937 loss: 21929.36307489872
epoch 2 total_correct: 51937 loss: 21929.36307489872
epoch 2 total_correct: 51937 loss: 21929.36307489872
epoch 2 total_correct: 51937 loss: 2

epoch 2 total_correct: 52444 loss: 20380.12653536396
epoch 2 total_correct: 52444 loss: 20380.12653536396
epoch 2 total_correct: 52444 loss: 20380.12653536396
epoch 2 total_correct: 52444 loss: 20380.12653536396
epoch 2 total_correct: 52444 loss: 20380.12653536396
epoch 2 total_correct: 52444 loss: 20380.12653536396
epoch 2 total_correct: 52444 loss: 20380.12653536396
epoch 2 total_correct: 52444 loss: 20380.12653536396
epoch 2 total_correct: 52444 loss: 20380.12653536396
epoch 2 total_correct: 52444 loss: 20380.12653536396
epoch 3 total_correct: 53002 loss: 18853.99706000462
epoch 3 total_correct: 53002 loss: 18853.99706000462
epoch 3 total_correct: 53002 loss: 18853.99706000462
epoch 3 total_correct: 53002 loss: 18853.99706000462
epoch 3 total_correct: 53002 loss: 18853.99706000462
epoch 3 total_correct: 53002 loss: 18853.99706000462
epoch 3 total_correct: 53002 loss: 18853.99706000462
epoch 3 total_correct: 53002 loss: 18853.99706000462
epoch 3 total_correct: 53002 loss: 18853.99706

epoch 3 total_correct: 50883 loss: 25176.48721933365
epoch 3 total_correct: 50883 loss: 25176.48721933365
epoch 3 total_correct: 50883 loss: 25176.48721933365
epoch 3 total_correct: 50883 loss: 25176.48721933365
epoch 3 total_correct: 50883 loss: 25176.48721933365
epoch 3 total_correct: 50883 loss: 25176.48721933365
epoch 3 total_correct: 50883 loss: 25176.48721933365
epoch 3 total_correct: 50883 loss: 25176.48721933365
epoch 3 total_correct: 50883 loss: 25176.48721933365
epoch 3 total_correct: 50883 loss: 25176.48721933365
epoch 4 total_correct: 51646 loss: 23061.67551279068
epoch 4 total_correct: 51646 loss: 23061.67551279068
epoch 4 total_correct: 51646 loss: 23061.67551279068
epoch 4 total_correct: 51646 loss: 23061.67551279068
epoch 4 total_correct: 51646 loss: 23061.67551279068
epoch 4 total_correct: 51646 loss: 23061.67551279068
epoch 4 total_correct: 51646 loss: 23061.67551279068
epoch 4 total_correct: 51646 loss: 23061.67551279068
epoch 4 total_correct: 51646 loss: 23061.67551

epoch 4 total_correct: 50540 loss: 26092.048059459776
epoch 4 total_correct: 50540 loss: 26092.048059459776
epoch 4 total_correct: 50540 loss: 26092.048059459776
epoch 4 total_correct: 50540 loss: 26092.048059459776
epoch 4 total_correct: 50540 loss: 26092.048059459776
epoch 4 total_correct: 50540 loss: 26092.048059459776
epoch 4 total_correct: 50540 loss: 26092.048059459776
epoch 4 total_correct: 50540 loss: 26092.048059459776
epoch 4 total_correct: 50540 loss: 26092.048059459776
epoch 4 total_correct: 50540 loss: 26092.048059459776
0.0001 10 False
epoch 0 total_correct: 40498 loss: 52173.60046334565
epoch 0 total_correct: 40498 loss: 52173.60046334565
epoch 0 total_correct: 40498 loss: 52173.60046334565
epoch 0 total_correct: 40498 loss: 52173.60046334565
epoch 0 total_correct: 40498 loss: 52173.60046334565
epoch 0 total_correct: 40498 loss: 52173.60046334565
epoch 0 total_correct: 40498 loss: 52173.60046334565
epoch 0 total_correct: 40498 loss: 52173.60046334565
epoch 0 total_correc

epoch 0 total_correct: 8155 loss: 137436.01727485657
epoch 0 total_correct: 8155 loss: 137436.01727485657
epoch 0 total_correct: 8155 loss: 137436.01727485657
epoch 0 total_correct: 8155 loss: 137436.01727485657
epoch 0 total_correct: 8155 loss: 137436.01727485657
epoch 0 total_correct: 8155 loss: 137436.01727485657
epoch 0 total_correct: 8155 loss: 137436.01727485657
epoch 0 total_correct: 8155 loss: 137436.01727485657
epoch 0 total_correct: 8155 loss: 137436.01727485657
epoch 0 total_correct: 8155 loss: 137436.01727485657
epoch 1 total_correct: 19422 loss: 130087.96834945679
epoch 1 total_correct: 19422 loss: 130087.96834945679
epoch 1 total_correct: 19422 loss: 130087.96834945679
epoch 1 total_correct: 19422 loss: 130087.96834945679
epoch 1 total_correct: 19422 loss: 130087.96834945679
epoch 1 total_correct: 19422 loss: 130087.96834945679
epoch 1 total_correct: 19422 loss: 130087.96834945679
epoch 1 total_correct: 19422 loss: 130087.96834945679
epoch 1 total_correct: 19422 loss: 130

epoch 1 total_correct: 39176 loss: 59261.379331201315
epoch 1 total_correct: 39176 loss: 59261.379331201315
epoch 1 total_correct: 39176 loss: 59261.379331201315
epoch 1 total_correct: 39176 loss: 59261.379331201315
epoch 1 total_correct: 39176 loss: 59261.379331201315
epoch 1 total_correct: 39176 loss: 59261.379331201315
epoch 1 total_correct: 39176 loss: 59261.379331201315
epoch 1 total_correct: 39176 loss: 59261.379331201315
epoch 1 total_correct: 39176 loss: 59261.379331201315
epoch 1 total_correct: 39176 loss: 59261.379331201315
epoch 2 total_correct: 41989 loss: 48960.36825016141
epoch 2 total_correct: 41989 loss: 48960.36825016141
epoch 2 total_correct: 41989 loss: 48960.36825016141
epoch 2 total_correct: 41989 loss: 48960.36825016141
epoch 2 total_correct: 41989 loss: 48960.36825016141
epoch 2 total_correct: 41989 loss: 48960.36825016141
epoch 2 total_correct: 41989 loss: 48960.36825016141
epoch 2 total_correct: 41989 loss: 48960.36825016141
epoch 2 total_correct: 41989 loss: 4

epoch 2 total_correct: 6243 loss: 137882.13324546814
epoch 2 total_correct: 6243 loss: 137882.13324546814
epoch 2 total_correct: 6243 loss: 137882.13324546814
epoch 2 total_correct: 6243 loss: 137882.13324546814
epoch 2 total_correct: 6243 loss: 137882.13324546814
epoch 2 total_correct: 6243 loss: 137882.13324546814
epoch 2 total_correct: 6243 loss: 137882.13324546814
epoch 2 total_correct: 6243 loss: 137882.13324546814
epoch 2 total_correct: 6243 loss: 137882.13324546814
epoch 2 total_correct: 6243 loss: 137882.13324546814
epoch 3 total_correct: 6163 loss: 137668.61534118652
epoch 3 total_correct: 6163 loss: 137668.61534118652
epoch 3 total_correct: 6163 loss: 137668.61534118652
epoch 3 total_correct: 6163 loss: 137668.61534118652
epoch 3 total_correct: 6163 loss: 137668.61534118652
epoch 3 total_correct: 6163 loss: 137668.61534118652
epoch 3 total_correct: 6163 loss: 137668.61534118652
epoch 3 total_correct: 6163 loss: 137668.61534118652
epoch 3 total_correct: 6163 loss: 137668.61534